# My Notebook for
## Cryptocurrency Closing Price Prediction Challenge


Can you predict the closing price for a cryptocurrency? 

The objective of this challenge is to create a machine learning model that will predict closing price for a crypto currency coin. 

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,BayesianRidge
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler,PolynomialFeatures
from google.colab import files
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn import pipeline

from sklearn.feature_selection import RFE
from itertools import combinations_with_replacement

from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load files
train = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')
ss = pd.read_csv('SampleSubmission.csv')

In [ ]:
# check data shapes
train.shape, test_df.shape, ss.shape

((12632, 49), (6222, 48), (6222, 2))

In [ ]:
train.describe()

,asset_id,open,high,low,volume,market_cap,url_shares,unique_url_shares,reddit_posts,reddit_posts_score,reddit_comments,reddit_comments_score,tweets,tweet_spam,tweet_followers,tweet_quotes,tweet_retweets,tweet_replies,tweet_favorites,tweet_sentiment1,tweet_sentiment2,tweet_sentiment3,tweet_sentiment4,tweet_sentiment5,tweet_sentiment_impact1,tweet_sentiment_impact2,tweet_sentiment_impact3,tweet_sentiment_impact4,tweet_sentiment_impact5,social_score,average_sentiment,news,price_score,social_impact_score,correlation_rank,galaxy_score,volatility,market_cap_rank,percent_change_24h_rank,volume_24h_rank,social_volume_24h_rank,social_score_24h_rank,medium,youtube,social_volume,percent_change_24h,market_cap_global,close
count,12632.0,8617.000000,8617.000000,8617.000000,8.605000e+03,8.379000e+03,8575.000000,8575.000000,12391.000000,12391.000000,12402.000000,12402.000000,8584.000000,8584.000000,8.574000e+03,8574.000000,8574.000000,8574.000000,8574.000000,8584.000000,8584.000000,8584.000000,8584.000000,8584.000000,8.584000e+03,8.584000e+03,8.584000e+03,8.584000e+03,8.584000e+03,8.584000e+03,8584.000000,8556.000000,8584.000000,8584.000000,8584.000000,8584.000000,8546.000000,8314.000000,8107.000000,8107.000000,8107.000000,8107.000000,2950.000000,4786.000000,12632.000000,8313.000000,8.351000e+03,8617.000000
mean,1.0,15504.816128,15559.484568,15449.591187,1.300724e+09,2.826010e+11,1247.310437,757.862857,36.527722,323.989831,101.397436,373.067328,4787.700839,1359.266193,2.240930e+07,152.879170,1673.725916,628.944717,5729.612783,135.003611,400.070480,675.888048,3336.687791,233.689306,1.414540e+06,2.870625e+06,2.658385e+06,1.403838e+07,1.730606e+06,2.263903e+07,3.654485,25.713651,3.002924,3.683644,3.269793,68.062698,0.015378,1.001323,809.518811,2.128037,1.018749,1.005921,1.349492,3.138947,3920.713268,0.375253,4.616824e+11,15507.395867
std,0.0,14105.710063,14191.106378,14022.651911,1.711922e+09,2.663052e+11,593.108269,303.627395,31.610276,677.419559,89.474521,2289.393857,4698.346982,1319.529420,2.925547e+07,825.124037,6897.963587,1131.808003,13515.879088,158.919580,453.837188,499.411760,3710.525848,218.058142,3.727319e+06,5.994027e+06,4.095968e+06,1.818741e+07,3.988111e+06,3.024835e+07,0.095395,23.701377,0.789208,0.579811,0.727227,6.237936,0.019247,0.066702,225.420505,0.432902,0.135646,0.076723,0.868552,5.470365,4824.053810,3.745955,4.597152e+11,14112.405285
min,1.0,4545.087790,4589.712024,4267.292258,2.373622e+08,8.393765e+10,2.000000,2.000000,1.000000,0.000000,1.000000,0.000000,218.000000,24.000000,6.965840e+05,0.000000,3.000000,0.000000,2.000000,0.000000,5.000000,23.000000,141.000000,4.000000,0.000000e+00,2.363000e+03,6.622000e+04,2.831590e+05,4.967000e+03,0.000000e+00,2.000000,1.000000,1.000000,1.000000,1.000000,37.000000,0.001580,1.000000,124.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-44.690403,1.223398e+11,4267.292258
25%,1.0,8405.552062,8437.391908,8378.513061,6.196589e+08,1.506098e+11,857.000000,559.000000,13.000000,79.000000,41.000000,128.000000,2602.500000,342.000000,9.311164e+06,33.000000,517.000000,197.000000,1490.000000,65.000000,143.000000,387.000000,1706.500000,118.000000,1.514345e+05,5.322950e+05,1.133224e+06,6.243698e+06,2.800610e+05,9.279215e+06,3.600000,10.000000,2.700000,3.000000,2.800000,64.500000,0.006975,1.000000,661.000000,2.000000,1.000000,1.000000,1.000000,1.000000,243.000000,-1.252658,2.265741e+11,8402.681367
50%,1.0,9671.831848,9689.751652,9648.410207,9.820621e+08,1.755833e+11,1124.000000,703.000000,28.000000,156.000000,65.000000,233.000000,3619.000000,811.000000,1.408329e+07,70.000000,881.000000,367.500000,2733.500000,102.000000,253.500000,557.000000,2423.000000,184.000000,3.352840e+05,1.111772e+06,1.710463e+06,9.478264e+06,5.413245e+05,1.408742e+07,3.700000,24.000000,3.000000,3.800000,3.200000,68.000000,0.011193,1.000000,819.000000,2.000000,1.000000,1.000000,1.000000,2.000000,3334.000000,0.240354,2.716129e+11,9667.590065
75%,1.0,13077.201438,13089.839678,13047.114786,1.611974e+09,2.191438e+11,1507.5

In [ ]:
#merging the training and testing set
all_data=pd.concat((train,test_df)).reset_index(drop=True)

In [ ]:
#I performed variable-wise droping using the function below
def drop_var(data):
  #let test this 

  #create a set of features which are not collinear #########

  data['poly']=data['high']+data['low']*data['galaxy_score']

  data['social_tweets']=data['tweets']*data['volatility']+data['social_score']*data['correlation_rank']

  data['galaxy_tweets']=data['galaxy_score']*data['volatility'] +data['high']

  data['poly_1']=data['low']+data['open']*data['galaxy_score']

  data['poly_2']=data['open']+data['high']*data['galaxy_score']

  data['poly_3']=data['high']+data['open']*data['galaxy_score']

  data['high_low_open']=data['high']*data['open']+data['low']*data['low']

  ###good factors #######

  data['good_fact']=data['volume']+data['tweets']+data['market_cap']

  data= data.drop(columns=['social_volume_24h_rank','percent_change_24h_rank','market_cap_rank','average_sentiment'
  ,'tweet_sentiment_impact4','tweet_sentiment_impact2','tweet_sentiment_impact1','tweet_sentiment4',
  'tweet_sentiment3','tweet_sentiment2','tweet_favorites','tweet_replies','tweet_quotes','tweet_spam','reddit_comments_score',
  'reddit_comments'])
  #rest of the market
  return data

In [ ]:
all_data=drop_var(all_data)

In [ ]:
#       ['id', 'asset_id', 'open', 'high', 'low', 'volume', 'market_cap',
#       'url_shares', 'unique_url_shares', 'reddit_posts', 'reddit_posts_score',
#       'tweets', 'tweet_followers', 'tweet_retweets', 'tweet_sentiment1',
#       'tweet_sentiment5', 'tweet_sentiment_impact3',
#       'tweet_sentiment_impact5', 'social_score', 'news', 'price_score',
#       'social_impact_score', 'correlation_rank', 'galaxy_score', 'volatility',
#       'volume_24h_rank', 'social_score_24h_rank', 'medium', 'youtube',
#       'social_volume', 'percent_change_24h', 'market_cap_global', 'close']

In [ ]:
#the first step is to perform feature interaction
all_cols = all_data.columns.difference(['id', 'close','asset_id','target_class'])
interact = [col for col in all_data[all_cols].columns if all_data[col].corr(all_data['close']) > 0.7]

In [ ]:
#I created a feature interaction function
def feature_interact(data,col):

  #creating dataframes
  df1=pd.DataFrame()

  df2=pd.DataFrame()

  df3=pd.DataFrame()

  df4=pd.DataFrame()


  for p in combinations_with_replacement(data[col].columns,2):

    ######## multiplication ##########

    title_1 = p + ("multiply",)
    df1[title_1] = data[col][p[0]]*data[col][p[1]] 

    ####  addition      #######

    title_2= p+ ("add",)
    df2[title_2]=data[col][p[0]]+data[col][p[1]]

    ##### subtraction  ##########

    title_3= p+("subtract",)
    df3[title_3]=data[col][p[0]]-data[col][p[1]]

    ##### divide   #############

    title_4= p+("divide",)
    df4[title_4]=data[col][p[0]]/data[col][p[1]]


  return pd.concat((data,df1,df2,df3,df4),axis=1)

In [ ]:
all_data=feature_interact(all_data,interact)

In [ ]:
# the division sign put in some infinity  values, so we remove them together with the NAN
all_data.replace([np.inf, -np.inf], np.nan, inplace=True)
all_data.fillna(0, inplace=True)

In [ ]:
train_size=train.shape[0]
##now we can split the data
train=all_data[:train_size]
test=all_data[train_size:]

In [ ]:
# fill missing values
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
test= test.drop('close', axis=1)
all_cols = train.columns.difference(['id', 'close'])

In [ ]:
# Select main columns to be used in training
main_cols = train.columns.difference(['id', 'close','asset_id','target_class'])
X = train[main_cols]
y = train.close.astype(float)

In [ ]:
col_1=[('high', 'high', 'add'),
       ('high', 'high', 'multiply'),
       ('high', 'low', 'add'),
       ('high', 'low', 'multiply'), ('high', 'low', 'subtract'),
       ('high', 'market_cap', 'add'), ('high', 'market_cap', 'divide'),
       ('high', 'market_cap', 'multiply'),
       ('high', 'market_cap', 'subtract'),
       ('high', 'market_cap_global', 'add'),
       ('high', 'market_cap_global', 'divide'),
       ('high', 'market_cap_global', 'multiply'),
       ('high', 'market_cap_global', 'subtract'), ('high', 'open', 'add'),
       ('high', 'open', 'multiply'),
        ('high', 'poly', 'add'),
       ('high', 'poly', 'multiply'),
       ('high', 'poly', 'subtract'), ('high', 'volume_24h_rank', 'add'),
       ('high', 'volume_24h_rank', 'divide'),
       ('high', 'volume_24h_rank', 'multiply'),
       ('high', 'volume_24h_rank', 'subtract'), ('low', 'low', 'add'),
        ('low', 'low', 'multiply'),
        ('low', 'market_cap', 'add'),
       ('low', 'market_cap', 'divide'), ('low', 'market_cap', 'multiply'),
       ('low', 'market_cap', 'subtract'),
       ('low', 'market_cap_global', 'add'),
       ('low', 'market_cap_global', 'divide'),
       ('low', 'market_cap_global', 'multiply'),
       ('low', 'market_cap_global', 'subtract'), ('low', 'open', 'add'),
       ('low', 'open', 'multiply'),
       ('low', 'poly', 'add'),
       ('low', 'poly', 'multiply'), ('low', 'poly', 'subtract'),
       ('low', 'volume_24h_rank', 'add'),
       ('low', 'volume_24h_rank', 'multiply'),
       ('low', 'volume_24h_rank', 'subtract'),
       ('market_cap', 'market_cap', 'add'),
       ('market_cap', 'market_cap', 'multiply'),
       ('market_cap', 'market_cap_global', 'add'),
       ('market_cap', 'market_cap_global', 'divide'),
       ('market_cap', 'market_cap_global', 'multiply'),
       ('market_cap', 'market_cap_global', 'subtract'),
       ('market_cap', 'open', 'add'), ('market_cap', 'open', 'divide'),
       ('market_cap', 'open', 'multiply'),
       ('market_cap', 'open', 'subtract'), ('market_cap', 'poly', 'add'),
       ('market_cap', 'poly', 'multiply'),
       ('market_cap', 'poly', 'subtract'),
       ('market_cap', 'volume_24h_rank', 'add'),
       ('market_cap', 'volume_24h_rank', 'multiply'),
       ('market_cap', 'volume_24h_rank', 'subtract'),
       ('market_cap_global', 'market_cap_global', 'add'),
       ('market_cap_global', 'open', 'add'),
       ('market_cap_global', 'open', 'multiply'),
       ('market_cap_global', 'open', 'subtract'),
       ('market_cap_global', 'poly', 'add'),
       ('market_cap_global', 'poly', 'subtract'),
       ('market_cap_global', 'volume_24h_rank', 'add'),
       ('market_cap_global', 'volume_24h_rank', 'subtract'),
       ('open', 'open', 'add'),
       ('open', 'open', 'multiply'), ('open', 'poly', 'add'),
        ('open', 'poly', 'multiply'),
       ('open', 'poly', 'subtract'), ('open', 'volume_24h_rank', 'add'),
       ('open', 'volume_24h_rank', 'multiply'),
       ('open', 'volume_24h_rank', 'subtract'), ('poly', 'poly', 'add'),
       ('poly', 'volume_24h_rank', 'add'),
       ('poly', 'volume_24h_rank', 'subtract'),
       ('volume_24h_rank', 'volume_24h_rank', 'add'), 'high', 'low',
       'market_cap', 'market_cap_global', 'open', 'poly',
       'volume_24h_rank','galaxy_tweets','poly_1','poly_2'
       ,'poly_3','high_low_open']

In [ ]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y):
    mi_scores = mutual_info_regression(X, y)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [ ]:
#mi_score=make_mi_scores(X[col_1],y)

In [ ]:
#mi_score[80:]

In [ ]:
col_2=[('high', 'high', 'add'),
       ('high', 'high', 'multiply'), ('high', 'low', 'add'),
        ('high', 'low', 'multiply'),
       ('high', 'market_cap', 'add'), 
       ('high', 'market_cap', 'multiply'),
       ('high', 'market_cap', 'subtract'),
       ('high', 'market_cap_global', 'add'),
       ('high', 'market_cap_global', 'multiply'),
       ('high', 'market_cap_global', 'subtract'), ('high', 'open', 'add'),
       ('high', 'open', 'multiply'),
       ('high', 'poly', 'add'),
       ('high', 'poly', 'multiply'), ('high', 'poly', 'subtract'),
       ('high', 'volume_24h_rank', 'add'),
       ('high', 'volume_24h_rank', 'divide'),
       ('high', 'volume_24h_rank', 'multiply'),
       ('high', 'volume_24h_rank', 'subtract'), ('low', 'low', 'add'),
       ('low', 'low', 'multiply'),
       ('low', 'market_cap', 'add'), 
       ('low', 'market_cap', 'multiply'),
       ('low', 'market_cap', 'subtract'),
       ('low', 'market_cap_global', 'add'),
       ('low', 'market_cap_global', 'multiply'),
       ('low', 'market_cap_global', 'subtract'), ('low', 'open', 'add'),
        ('low', 'open', 'multiply'),
       ('low', 'poly', 'add'), 
       ('low', 'poly', 'multiply'), ('low', 'poly', 'subtract'),
       ('low', 'volume_24h_rank', 'add'),
       ('low', 'volume_24h_rank', 'divide'),
       ('low', 'volume_24h_rank', 'multiply'),
       ('low', 'volume_24h_rank', 'subtract'),
       ('market_cap', 'market_cap', 'add'),
       ('market_cap', 'market_cap', 'multiply'),
       ('market_cap', 'market_cap_global', 'add'),
       ('market_cap', 'market_cap_global', 'multiply'),
       ('market_cap', 'market_cap_global', 'subtract'),
       ('market_cap', 'open', 'add'), 
       ('market_cap', 'open', 'multiply'),
       ('market_cap', 'open', 'subtract'), ('market_cap', 'poly', 'add'),
       ('market_cap', 'poly', 'subtract'),
       ('market_cap', 'volume_24h_rank', 'divide'),
       ('market_cap', 'volume_24h_rank', 'multiply'),
       ('market_cap_global', 'market_cap_global', 'add'),
       
       ('market_cap_global', 'market_cap_global', 'multiply'),
       ('market_cap_global', 'open', 'add'),
       ('market_cap_global', 'open', 'divide'),
       ('market_cap_global', 'open', 'multiply'),
       ('market_cap_global', 'open', 'subtract'),
       ('market_cap_global', 'poly', 'add'),
       ('market_cap_global', 'poly', 'multiply'),
       ('market_cap_global', 'poly', 'subtract'),
       ('market_cap_global', 'volume_24h_rank', 'add'),
       ('market_cap_global', 'volume_24h_rank', 'divide'),
       ('market_cap_global', 'volume_24h_rank', 'multiply'),
       ('market_cap_global', 'volume_24h_rank', 'subtract'),
       ('open', 'open', 'add'),
       ('open', 'open', 'multiply'), ('open', 'poly', 'add'),
        ('open', 'poly', 'multiply'),
       ('open', 'poly', 'subtract'), ('open', 'volume_24h_rank', 'add'),
       ('open', 'volume_24h_rank', 'divide'),
       ('open', 'volume_24h_rank', 'multiply'),
       ('open', 'volume_24h_rank', 'subtract'), ('poly', 'poly', 'add'),
        'high', 'low',
       'market_cap', 'market_cap_global', 'open', 'poly','galaxy_tweets','poly_1','poly_2',
       'high_low_open','poly_3',
('poly', 'volume_24h_rank', 'subtract'),
('poly', 'volume_24h_rank', 'add'),         
('high', 'market_cap_global', 'divide'),         
('low', 'market_cap_global', 'divide'),           
('poly', 'volume_24h_rank', 'multiply'),          
('market_cap', 'market_cap_global', 'divide'),    
('poly', 'volume_24h_rank', 'divide'),            
('market_cap_global', 'open', 'divide'),          
('low', 'market_cap', 'divide'),                  
('high', 'market_cap', 'divide'),                 
('market_cap', 'open', 'divide'),                
('low', 'open', 'divide')]

In [ ]:
#mi_score=make_mi_scores(X[col_2],y)

In [ ]:
#model = RFE(estimator = BayesianRidge(), n_features_to_select = 100)
#model.fit(X_train,y_train)
col=[col_1,col_2]

In [ ]:
#now I create a stacking model
Linear_regressors=[LinearRegression(),BayesianRidge()] #

def stack_model(models,X,y,col,test_data,test_df):

  

  #creating a list to append the values of predictions of training and testing in each case

  train_prediction = []
  test_prediction = []
  pred = []
  scores=[]

  #We now decide to fit differrent models

  for i,model in  enumerate(models):


    sc = RobustScaler()

    ## splitting into train and validation
    X_train = X[col[i]].loc[:9500]
    X_test = X[col[i]].loc[9500:]
    y_train = y.loc[:9500]
    y_test = y.loc[9500:]
    test=test_data[col[i]]

    ###preprocessing#############

    sc.fit(X_test)
    X_train = sc.transform(X_train)

    X_test = sc.transform(X_test)
    test = sc.transform(test)


    #fitting the models

    #model = RFE(estimator = model , n_features_to_select = 100)
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    train_predt = model.predict(test)    
  
    #computing the rmse for both training and testing

    test_rmse= mean_squared_error(y_test, test_pred, squared=False)
    train_rmse = mean_squared_error(y_train, train_pred, squared=False)
    train_prediction.append(train_pred)
    test_prediction.append(test_pred)
    pred.append(train_predt)



    print(model, "train rmse:", train_rmse, "test rmse:", test_rmse)
    scores.append(test_rmse)
  print("Average rmse:",np.mean(scores))

  #now we create an empty dataframe to fit in the predicted output

  submit=pd.DataFrame()
  submit['id'] = test_df['id']
  submit['pred'] = np.mean(pred, axis=0)

  return submit

In [ ]:
submit=stack_model(Linear_regressors,X,y,col,test[main_cols],test_df)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False) train rmse: 38.55935423338109 test rmse: 46.26442591784489
BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False) train rmse: 38.4363622988337 test rmse: 41.914413649162896
Average rmse: 44.08941978350389


In [ ]:
submit['pred']=submit['pred'].apply(lambda x: 0 if x<10 else x)

In [ ]:
#now we can 
#n=100 41.87497004207674

In [ ]:
submit.head()

,id,pred
0,ID_323Sn2,0.000000
1,ID_325SNW,11346.556511
2,ID_325uzE,6311.748574
3,ID_328qCx,0.000000
4,ID_3293uJ,0.000000


we have noticed that when ever the high and the low price or high and open price or open and low price is thesame, the closing price is the other price, so we used this idea to scale our outputs, by averaging them (It doesn't whole for every trading activity)

In [ ]:
def kubrah_function(data,exp):

  #the function to compare the predicted close price and open,low and high price

  for i,j in enumerate(exp):

    ##compare low and close

    if exp.loc[i,'close_low']<2:

      data.loc[i,'pred']=exp.loc[i,'low']

    ## compare high and close

    elif exp.loc[i,'close_high']<2:

      data.loc[i,'pred']=exp.loc[i,'high']

    ## compare open and close

    elif exp.loc[i,'open_close']<2:

      data.loc[i,'pred']=exp.loc[i,'open']

  ## return the function ####

  return data

In [ ]:
test_df['close']=submit.pred

The first step is to create a dataframe containing the high,low,open and predicted close price of the test set. Next I compute the difference between *(high,close), (low,close) and (open,close)*.
1. Base on the previous hypothesis if the difference between, high and close is small then the closing price is probably the high price.
1. If the difference between, low and close is small then the closing price is probably the low price.
1. If the difference between, open and close is small then the closing price is probably the open price.

In [ ]:
exp=test_df[['open','high','low','close']]
exp['open_close']=np.abs(exp['open']-exp['close'])
exp['close_high']=np.abs(exp['close']-exp['high'])
exp['close_low']=np.abs(exp['close']-exp['low'])

In [ ]:
# use the kubrah_function
submit=kubrah_function(submit,exp)

In [ ]:
submit.head()

,id,pred
0,ID_323Sn2,0.000000
1,ID_325SNW,11346.556511
2,ID_325uzE,6311.748574
3,ID_328qCx,0.000000
4,ID_3293uJ,0.000000


In [ ]:
submit.to_csv('kubrah.csv',index=False) 